# Exploratory Data Analysis

This file will have the intention to check vouchers data, en determine which will. be the transformations needed to take in place in the ETL.

Let's start with the simplier, importing Pandas lib. Pandas is a data analytics tool that let us check data structure in Dataframe-fashion. Is a easy to implement tool, open source and realiable tool, that will help to explore data before threat it.

In [2]:
import pandas as pd
import numpy as np

Reading data from `gzip` file.

Also, in the first step, let's filter data available just for Peru. As part of the requeriment is to build a solution for Peru vouchers, I'm not in the need to analyse the rest of countries by this moment.

In [3]:
data_df = pd.read_parquet("data/input/data.parquet.gzip")
peru_df = data_df.query('country_code=="Peru"')

Let's take a quick view of the data

In [4]:
peru_df.head(15)

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
5,2020-05-20 15:24:04.621986+00:00,Peru,2020-04-19 00:00:00+00:00,2017-07-24 00:00:00+00:00,2.0,2640.0
39,2020-05-20 14:27:43.083902+00:00,Peru,2020-04-19 00:00:00+00:00,2018-06-19 00:00:00+00:00,20.0,NaN
63,2020-05-20 15:00:18.431343+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-13 00:00:00+00:00,27.0,2640.0
81,2017-03-20 15:42:38.570961+00:00,Peru,2020-04-19 00:00:00+00:00,2019-05-21 00:00:00+00:00,10.0,4400.0
103,2020-05-20 14:31:34.507740+00:00,Peru,2020-04-19 00:00:00+00:00,2019-11-01 00:00:00+00:00,27.0,NaN
108,2020-05-20 15:43:18.972546+00:00,Peru,2020-04-19 00:00:00+00:00,2019-11-05 00:00:00+00:00,38.0,4400.0
109,2020-05-20 15:06:20.855401+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-14 00:00:00+00:00,39.0,2640.0
110,2020-05-20 15:05:25.186604+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-25 00:00:00+00:00,20.0,2640.0
112,2020-05-20 15:00:21.245225+00:00,Peru,2020-04-19 00:00:00+00:00,2019-06-26 00:00:00+00:00,11.0,2640.0
115,2020-05-20 15:25:17.065148+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-19 00:00:00+00:00,,2640.0


A quick first comment can be, there are some issues over `total_orders` and `voucher_amount` columns, coming with some null or empty values. Let's dig more into the dataset.

To have a better perspective of data, let's see how many vouchers are coming from Peru:

In [5]:
peru_df.shape[0]

106547

106k Records seem to be a good sample of data for exploratory porpuses. As a next step, I will print some basic profiling counts:

In [6]:
peru_df.count()

timestamp         106547
country_code      106547
last_order_ts     106547
first_order_ts    106547
total_orders      106547
voucher_amount     92597
dtype: int64

As we saw before, there are some issues with `voucher_amount` column. Not all the vourchers coming in the sample, are coming with data in the `voucher_amount` (13950 vouchers). The missing data represents a 13.09% percent of the sample. A significant number. 

The voucher amount, is an important metric in the dataset, as that will be the value returned by the API every time this one has been requested, so having vouchers with null values in the final dataset may lead to a bad perspective of the data.

The API should return the most used value for the voucher, so, asumme any default value for null voucher can also be consider as an irresposnable solution, due to there is no driver that can guide into a valide assumtion. Based on that, the best solution to be taken, is to drop `NaN` vourchers from dataset

In [7]:
peru_df_not_null = peru_df.dropna()
peru_df_not_null.count()

timestamp         92597
country_code      92597
last_order_ts     92597
first_order_ts    92597
total_orders      92597
voucher_amount    92597
dtype: int64

Now, let's move forward to check the rest of the data. Let's see data types of each column:

In [8]:
peru_df_not_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92597 entries, 5 to 511424
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       92597 non-null  object             
 1   country_code    92597 non-null  object             
 2   last_order_ts   92597 non-null  object             
 3   first_order_ts  92597 non-null  datetime64[ns, UTC]
 4   total_orders    92597 non-null  object             
 5   voucher_amount  92597 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 4.9+ MB


total_orders columns seems to be filled without any null value, but in the first look of the data it shows some empty data. I will definetelly dig over there in a moment. Also, the data type of the column is an object, that means that some data can be there without number format.

Also, columns timestamp and last_order_ts are represented as objects. This can be dangerous as there can be some values with incorrect tada formats. Let's also check that later.

Before take any action there, let's check for duplicated data:

In [9]:
peru_df_not_null[peru_df_not_null.duplicated(keep=False)]

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
326660,2020-04-21 17:02:22.915765+00:00,Peru,2020-01-22 00:00:00+00:00,2020-01-22 00:00:00+00:00,136.0,4400.0
326685,2020-04-21 17:02:22.915765+00:00,Peru,2020-01-22 00:00:00+00:00,2020-01-22 00:00:00+00:00,136.0,4400.0
326693,2020-04-21 17:04:55.965673+00:00,Peru,2020-01-22 00:00:00+00:00,2020-01-22 00:00:00+00:00,136.0,4400.0
326804,2020-04-21 17:04:55.965673+00:00,Peru,2020-01-22 00:00:00+00:00,2020-01-22 00:00:00+00:00,136.0,4400.0


Two rows are duplicated over more than 92k rows, doesn't seems like a big deal. But in order to keep data as clean as possible, I will drop that duplicated record:

In [10]:
peru_df_dedup = peru_df_not_null.drop_duplicates(keep='first')
peru_df_dedup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92595 entries, 5 to 511424
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       92595 non-null  object             
 1   country_code    92595 non-null  object             
 2   last_order_ts   92595 non-null  object             
 3   first_order_ts  92595 non-null  datetime64[ns, UTC]
 4   total_orders    92595 non-null  object             
 5   voucher_amount  92595 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 4.9+ MB


Now that duplicates have been removed, let's check `total_orders` column with empty values:

In [11]:
total_orders = peru_df_dedup['total_orders'].unique()
print(sorted(total_orders))

['', '0.0', '1.0', '10.0', '100.0', '101.0', '102.0', '103.0', '104.0', '105.0', '106.0', '107.0', '108.0', '109.0', '11.0', '110.0', '111.0', '112.0', '113.0', '114.0', '115.0', '116.0', '117.0', '118.0', '119.0', '12.0', '120.0', '121.0', '122.0', '123.0', '124.0', '125.0', '126.0', '127.0', '128.0', '129.0', '13.0', '130.0', '132.0', '133.0', '134.0', '135.0', '136.0', '137.0', '138.0', '139.0', '14.0', '141.0', '142.0', '143.0', '144.0', '145.0', '146.0', '147.0', '148.0', '149.0', '15.0', '150.0', '151.0', '152.0', '153.0', '154.0', '155.0', '156.0', '157.0', '158.0', '159.0', '16.0', '162.0', '164.0', '165.0', '166.0', '167.0', '168.0', '169.0', '17.0', '170.0', '172.0', '173.0', '174.0', '175.0', '176.0', '177.0', '178.0', '179.0', '18.0', '181.0', '182.0', '183.0', '184.0', '185.0', '187.0', '188.0', '189.0', '19.0', '190.0', '191.0', '192.0', '193.0', '194.0', '195.0', '197.0', '199.0', '2.0', '20.0', '200.0', '202.0', '203.0', '204.0', '205.0', '206.0', '207.0', '208.0', '209

In [12]:
peru_df_dedup.query('total_orders==""')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
115,2020-05-20 15:25:17.065148+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-19 00:00:00+00:00,,2640.0
165,2020-05-20 15:35:05.191404+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-16 00:00:00+00:00,,2640.0
187,2017-03-20 15:41:26.531505+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-18 00:00:00+00:00,,3520.0
329,2020-05-20 15:38:55.252756+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-30 00:00:00+00:00,,4400.0
602,2020-05-20 15:21:33.227664+00:00,Peru,2020-04-19 00:00:00+00:00,2019-06-09 00:00:00+00:00,,2640.0
...,...,...,...,...,...,...
510814,2020-04-03 18:49:54.319591+00:00,Peru,2020-01-04 00:00:00+00:00,2019-12-24 00:00:00+00:00,,4400.0
510928,2020-04-03 19:21:14.777117+00:00,Peru,2020-01-04 00:00:00+00:00,2019-07-02 00:00:00+00:00,,4400.0
510972,2020-04-03 19:11:23.273971+00:00,Peru,2020-01-04 00:00:00+00:00,2018-07-12 00:00:00+00:00,,4400.0
511386,2020-04-03 19:15:35.356191+00:00,Peru,2020-01-04 00:00:00+00:00,2019-01-16 00:00:00+00:00,,4400.0


`total_orders` columns is another important column to aim the main goal of the API. This column will be used to create the `frequent_segment` segment.

7447 is an important number. It represents the 8% of the data sample.

The number of total orders can't be infered for all the cases, but let's check a case. If the fisrt and last order date is the same for a single voucher, that means that for that specific line, there was only 1 order placed. In those cases, I could update the total_orders column:

In [13]:
peru_df_dedup.query('total_orders=="" & last_order_ts==first_order_ts')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
2567,2020-03-21 18:52:47.265898+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,,3520.0
2911,2020-03-21 18:50:53.022267+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,,2640.0
2969,2020-03-21 18:58:34.818910+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,,3520.0
3332,2020-03-21 18:50:37.504324+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,,4400.0
3339,2020-03-21 18:52:31.829822+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,,2640.0
...,...,...,...,...,...,...
509236,2020-04-03 19:06:06.869797+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,,4400.0
509255,2020-04-03 19:21:15.242708+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,,4400.0
509694,2020-04-03 19:19:31.476103+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,,2640.0
509724,2020-04-03 19:14:21.284921+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,,4400.0


1246 columns could be updated. But before take this desition, let's check how data behave when both dates are the same:

In [14]:
peru_df_dedup.query('last_order_ts==first_order_ts')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
1767,2020-05-20 15:38:05.268675+00:00,Peru,2020-04-19 00:00:00+00:00,2020-04-19 00:00:00+00:00,0.0,3520.0
2146,2020-03-21 19:03:20.505711+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,0.0,3520.0
2152,2020-03-21 18:50:02.050224+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,0.0,4400.0
2162,2020-03-21 18:59:27.378543+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,0.0,3520.0
2170,2020-03-21 18:49:37.037127+00:00,Peru,2019-12-20 00:00:00+00:00,2019-12-20 00:00:00+00:00,0.0,3520.0
...,...,...,...,...,...,...
509960,2020-04-03 19:11:55.946638+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,136.0,4400.0
509968,2020-04-03 19:17:49.984295+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,136.0,2640.0
509981,2020-04-03 19:17:11.096910+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,136.0,2640.0
510894,2020-04-03 19:12:32.733948+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-04 00:00:00+00:00,136.0,3520.0


In [15]:
total_orders_same_dates = peru_df_dedup.query('last_order_ts==first_order_ts')['total_orders'].unique()
print(sorted(total_orders_same_dates))

['', '0.0', '1.0', '136.0', '2.0', '27.0', '3.0', '37.0', '42.0', '47.0', '48.0']


It seems my initial hypotesis couldn't be followed. Transactions with same first and last order date not always place just 1 order. There are some several options that can be taken, that makes irresponsible my idea to place a `1`as the `total_orders`. So, this decision won't be taken.

Another hypotesis can be to place a `0` to all empty `total_orders`. But let's check how this behave:

In [17]:
peru_df_dedup.query('total_orders=="0.0"')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
179,2020-05-20 15:45:54.124568+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-17 00:00:00+00:00,0.0,4400.0
180,2020-05-20 15:44:51.557340+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-19 00:00:00+00:00,0.0,4400.0
183,2020-05-20 15:45:10.353056+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-19 00:00:00+00:00,0.0,2640.0
185,2020-05-20 15:41:29.356483+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-19 00:00:00+00:00,0.0,4400.0
1495,2020-05-20 15:39:12.983157+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-19 00:00:00+00:00,0.0,2640.0
...,...,...,...,...,...,...
510934,2020-04-03 19:22:08.166707+00:00,Peru,2020-01-04 00:00:00+00:00,2019-06-14 00:00:00+00:00,0.0,3520.0
510948,2020-04-03 18:56:59.460396+00:00,Peru,2020-01-04 00:00:00+00:00,2019-10-14 00:00:00+00:00,0.0,2640.0
511000,2020-04-03 19:22:55.090223+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-02 00:00:00+00:00,0.0,4400.0
511284,2020-04-03 19:14:30.057682+00:00,Peru,2020-01-04 00:00:00+00:00,2020-01-02 00:00:00+00:00,0.0,3520.0


Is not easy to define a rule here. There are several transactions with 0 orders. Without having an entire knowledge about business rules implied into data source generation, I would say transactions with values placed on first or last order ts is a mistake, as how can you have 0 orders if you already registered a fisrt order date and/or a last order date? 

But, checking again the Acceptance Criteria of this project, for segment `frequent_segment` definition, there is a value `0-4`. This means that having 0 orders transactions is possible. 

Probably there is a business rule I'm missing, so leading by prundet decisions, and trying to not make irresponsible assumptions over the data, I will drop all rows with emty `total_vouchers`, as no data can be infered.

In [19]:
peru_df_dedup.query('total_orders==""')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
115,2020-05-20 15:25:17.065148+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-19 00:00:00+00:00,,2640.0
165,2020-05-20 15:35:05.191404+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-16 00:00:00+00:00,,2640.0
187,2017-03-20 15:41:26.531505+00:00,Peru,2020-04-19 00:00:00+00:00,2020-03-18 00:00:00+00:00,,3520.0
329,2020-05-20 15:38:55.252756+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-30 00:00:00+00:00,,4400.0
602,2020-05-20 15:21:33.227664+00:00,Peru,2020-04-19 00:00:00+00:00,2019-06-09 00:00:00+00:00,,2640.0
...,...,...,...,...,...,...
510814,2020-04-03 18:49:54.319591+00:00,Peru,2020-01-04 00:00:00+00:00,2019-12-24 00:00:00+00:00,,4400.0
510928,2020-04-03 19:21:14.777117+00:00,Peru,2020-01-04 00:00:00+00:00,2019-07-02 00:00:00+00:00,,4400.0
510972,2020-04-03 19:11:23.273971+00:00,Peru,2020-01-04 00:00:00+00:00,2018-07-12 00:00:00+00:00,,4400.0
511386,2020-04-03 19:15:35.356191+00:00,Peru,2020-01-04 00:00:00+00:00,2019-01-16 00:00:00+00:00,,4400.0


This 7447 rows will be drop from dataset.

In [20]:
peru_df_orders_upd = peru_df_dedup.query('total_orders!=""')
peru_df_orders_upd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85148 entries, 5 to 511424
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       85148 non-null  object             
 1   country_code    85148 non-null  object             
 2   last_order_ts   85148 non-null  object             
 3   first_order_ts  85148 non-null  datetime64[ns, UTC]
 4   total_orders    85148 non-null  object             
 5   voucher_amount  85148 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 4.5+ MB


In [21]:
peru_df_orders_upd.head(15)

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
5,2020-05-20 15:24:04.621986+00:00,Peru,2020-04-19 00:00:00+00:00,2017-07-24 00:00:00+00:00,2.0,2640.0
63,2020-05-20 15:00:18.431343+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-13 00:00:00+00:00,27.0,2640.0
81,2017-03-20 15:42:38.570961+00:00,Peru,2020-04-19 00:00:00+00:00,2019-05-21 00:00:00+00:00,10.0,4400.0
108,2020-05-20 15:43:18.972546+00:00,Peru,2020-04-19 00:00:00+00:00,2019-11-05 00:00:00+00:00,38.0,4400.0
109,2020-05-20 15:06:20.855401+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-14 00:00:00+00:00,39.0,2640.0
110,2020-05-20 15:05:25.186604+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-25 00:00:00+00:00,20.0,2640.0
112,2020-05-20 15:00:21.245225+00:00,Peru,2020-04-19 00:00:00+00:00,2019-06-26 00:00:00+00:00,11.0,2640.0
116,2020-05-20 15:27:44.497849+00:00,Peru,2020-04-19 00:00:00+00:00,2020-02-01 00:00:00+00:00,20.0,2640.0
120,2020-05-20 15:25:30.640546+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-24 00:00:00+00:00,13.0,2640.0
125,2020-05-20 15:06:35.639553+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-17 00:00:00+00:00,4.0,2640.0


Data starts to look more organized and cleaner. Now, let's cast `total_orders` field to integer type. Let's check if all the data is in integer style:

In [22]:
peru_df_orders_upd['total_orders'].astype(str).str.split('.').str[1].unique()

array(['0'], dtype=object)

Splitting data from decimal point, just there are 0 values. This means `total_orders` column can be casted to `int`type.

In [23]:
peru_df_orders_cast = peru_df_orders_upd.astype({'total_orders':'float'}).astype({'total_orders':'int'})
peru_df_orders_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85148 entries, 5 to 511424
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       85148 non-null  object             
 1   country_code    85148 non-null  object             
 2   last_order_ts   85148 non-null  object             
 3   first_order_ts  85148 non-null  datetime64[ns, UTC]
 4   total_orders    85148 non-null  int64              
 5   voucher_amount  85148 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3)
memory usage: 4.5+ MB


In [24]:
peru_df_orders_cast.head(15)

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
5,2020-05-20 15:24:04.621986+00:00,Peru,2020-04-19 00:00:00+00:00,2017-07-24 00:00:00+00:00,2,2640.0
63,2020-05-20 15:00:18.431343+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-13 00:00:00+00:00,27,2640.0
81,2017-03-20 15:42:38.570961+00:00,Peru,2020-04-19 00:00:00+00:00,2019-05-21 00:00:00+00:00,10,4400.0
108,2020-05-20 15:43:18.972546+00:00,Peru,2020-04-19 00:00:00+00:00,2019-11-05 00:00:00+00:00,38,4400.0
109,2020-05-20 15:06:20.855401+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-14 00:00:00+00:00,39,2640.0
110,2020-05-20 15:05:25.186604+00:00,Peru,2020-04-19 00:00:00+00:00,2019-12-25 00:00:00+00:00,20,2640.0
112,2020-05-20 15:00:21.245225+00:00,Peru,2020-04-19 00:00:00+00:00,2019-06-26 00:00:00+00:00,11,2640.0
116,2020-05-20 15:27:44.497849+00:00,Peru,2020-04-19 00:00:00+00:00,2020-02-01 00:00:00+00:00,20,2640.0
120,2020-05-20 15:25:30.640546+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-24 00:00:00+00:00,13,2640.0
125,2020-05-20 15:06:35.639553+00:00,Peru,2020-04-19 00:00:00+00:00,2020-01-17 00:00:00+00:00,4,2640.0


Now that `total_columns` data was cleaned, let's check dates. First thing I will place, is to cast `last_order_ts` column to datetime type.

In [25]:
pd.to_datetime(peru_df_orders_cast['last_order_ts'], utc=True)

5        2020-04-19 00:00:00+00:00
63       2020-04-19 00:00:00+00:00
81       2020-04-19 00:00:00+00:00
108      2020-04-19 00:00:00+00:00
109      2020-04-19 00:00:00+00:00
                    ...           
511405   2020-01-04 00:00:00+00:00
511406   2020-01-04 00:00:00+00:00
511415   2020-01-04 00:00:00+00:00
511421   2020-01-04 00:00:00+00:00
511424   2020-01-04 00:00:00+00:00
Name: last_order_ts, Length: 85148, dtype: datetime64[ns, UTC]

It seems casting was mad successfully, so let's update the dataset.

timestamp column, won't be used in any of the transformations or for API porpuses, but in order to keep all data aligned, I also will casted it.

In [26]:
peru_df_date_cast = peru_df_orders_cast
peru_df_date_cast['last_order_ts'] = pd.to_datetime(peru_df_date_cast['last_order_ts'], utc=True)
peru_df_date_cast['timestamp'] = pd.to_datetime(peru_df_date_cast['timestamp'], utc=True)
peru_df_date_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85148 entries, 5 to 511424
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       85148 non-null  datetime64[ns, UTC]
 1   country_code    85148 non-null  object             
 2   last_order_ts   85148 non-null  datetime64[ns, UTC]
 3   first_order_ts  85148 non-null  datetime64[ns, UTC]
 4   total_orders    85148 non-null  int64              
 5   voucher_amount  85148 non-null  float64            
dtypes: datetime64[ns, UTC](3), float64(1), int64(1), object(1)
memory usage: 4.5+ MB


The last check I will take in place on dates, will be a comparisson between last and first date. Last date must be greater or equals than first date in all the rows. If this condition is not met, this can lead a bad analysis. Let's check that data:

In [27]:
peru_df_date_cast.query('last_order_ts<first_order_ts')

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount


Dates seems to be okay. So no transformation will be needed here.

Now, let's take a look over `voucher_amount` column. Let's see unique values to have a better perspective of data:

In [28]:
peru_df_date_cast['voucher_amount'].unique()

array([2640., 4400., 3520.])

The final set seems to be short and clean. All vourcher amounts seems to be valid, so no transformation will be needed here. I won't cast the values to `int` type as the `voucher_amount` represents money, so in the future prices with decimals can be placed.

Thats it! `peru_df_date_cast`get the most legit and trusted information about vouchers, that will be implemented in the API. Over this dataframe, the segments will be calculated into our data pipeline.

In [29]:
peru_df_date_cast.shape

(85148, 6)

A final outcome of 85k over an initial dataset of 106k is still a really great sample. It represents a 80% of the initial data, with the improvements and advantages that is trusted and realiable information. This final dataset will be used for the calculation of segments.

## Cleaning Transformations needed

After checking and profiling the data source, the following transformations will be needed for cleaning porpuses before start with segments transfromations:
1. To meet part of the Acceptance Criteria of the project, filter dataset by `country_code` equals to `Peru`
1. Drop rows with null values 
1. Drop duplicated rows
1. Drop rows with `total_orders` column with empty values
1. Cast `total_orders` column to `int` type
1. Cast `last_order_ts` and `timestamp` columns to `datetime64[ns, UTC]` type

Now these transformations had taken place over our initial dataset, we are ready to calculate segments and consume it from API